In [1]:
#dependencies
import re
from bs4 import BeautifulSoup 
from splinter import Browser
import requests
import pandas as pd
from selenium import webdriver 
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as BraveService
from webdriver_manager.core.utils import ChromeType
import json
import time
from tqdm.notebook import tqdm 

<h4> Start cells here if partial FIPS already available in {state}_fips_scraped file

In [2]:
fips_check_scraped = pd.read_csv("../../../data/state_data/geo/geo_fips/23/VA_fips_scraped.csv") 
fips_check_scraped  = fips_check_scraped.loc[:, ~fips_check_scraped.columns.str.contains('^Unnamed')] 
fips_check_scraped.head() 

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,2392497107,VA,"831 INDUSTRIAL AVE, CHESAPEAKE, VA",36.824257,-76.272486,5.155002e+14
1,2649457102,VA,"8827 STAPLES MILL RD, HENRICO, VA",37.633918,-77.507746,5.108720e+14
2,2952437110,VA,"1001 Boulders Parkway Ste 300, RICHMOND, VA",37.511077,-77.528711,5.104110e+14
3,3379937304,VA,"224 D Cornwall Street Suite 403, LEESBURG, VA",39.119430,-77.569008,5.110761e+14
4,3773127208,VA,"2242 W GREAT NECK RD, VIRGINIA BEACH, VA",36.909731,-76.070420,5.181004e+14


In [3]:
#Latitude, Longitude, not included
fips_check_scraped_notna = fips_check_scraped[fips_check_scraped["Lat"].notna()] # check to make sure no nulls in coords
fips_check_scraped_notna.head() 

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,2392497107,VA,"831 INDUSTRIAL AVE, CHESAPEAKE, VA",36.824257,-76.272486,5.155002e+14
1,2649457102,VA,"8827 STAPLES MILL RD, HENRICO, VA",37.633918,-77.507746,5.108720e+14
2,2952437110,VA,"1001 Boulders Parkway Ste 300, RICHMOND, VA",37.511077,-77.528711,5.104110e+14
3,3379937304,VA,"224 D Cornwall Street Suite 403, LEESBURG, VA",39.119430,-77.569008,5.110761e+14
4,3773127208,VA,"2242 W GREAT NECK RD, VIRGINIA BEACH, VA",36.909731,-76.070420,5.181004e+14


In [4]:
#FIPS not included, of Lat, Long available
fips_check_scraped_null = fips_check_scraped_notna[fips_check_scraped_notna["FIPS_z"].isnull()] # check to make sure no nulls in coords
fips_check_scraped_null #28093

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
291,4705037110,VA,"7918 Jones Branch Drive, Suite 300, McLean, VA",38.925933,-77.216943,NaN
326,6940727104,VA,"4625 Creekstone Drive Suite 130, OAKPARK, VA",35.882469,-78.846699,NaN
722,1708438404,VA,"7918 Jones Branch Dr Ste 300, McLean, VA",38.925933,-77.216943,NaN
1246,2219537102,VA,"5534 North MAIN ST, MOUNT JACKSON, VA",38.754798,-78.633557,NaN
2469,3320167707,VA,"777 6th Street NW, WASHINGTON, VA",38.899370,-77.019590,NaN
2653,5527667700,VA,"8200 GREENSBORO DR STE 900, MC LEAN, VA",38.922310,-77.227248,NaN
2705,3005707104,VA,"8200 Greensboro Dr, Suite 900, MC LEAN, VA",38.922310,-77.227248,NaN
2927,8472907107,VA,"5534 N MAIN ST, MOUNT JACKSON, VA",38.754798,-78.633557,NaN
3102,2188557104,VA,"5534 North MAIN ST, MOUNT JACKSON, VA",38.754798,-78.633557,NaN
3602,1804527302,VA,"8200 Greensboro Drive, Ste 900, MC LEAN, VA",38.922310,-77.227248,NaN


<h4>FIPS Script

In [5]:
#if FIPS_z file 
lat = fips_check_scraped_null["Lat"].astype(float).astype(str).to_list()
long = fips_check_scraped_null["Long"].astype(float).astype(str).to_list()

In [6]:
#before running script, check for any NaN within Lat, Long fields. 
def sleep():
    time.sleep(1)
option = webdriver.ChromeOptions()
option.binary_location = "/Applications/Brave Browser.app/Contents/MacOS/Brave Browser"
driver = Browser('chrome', options=option, headless=False)    
url = 'https://geocoding.geo.census.gov/geocoder/geographies/coordinates?form'
driver.visit(url)

not_found = []
geo_id = []

i=0
for i in tqdm(range(len(lat))):
    #while i < len(lat):
        driver.find_by_id('geoCoordX').clear()
        driver.find_by_id('geoCoordY').clear()
        driver.find_by_id('geoCoordX').fill(long[i])
        driver.find_by_id('geoCoordY').fill(lat[i])
        driver.find_by_id('geoCoordsSubmit').click()
        #sleep()
        #html = driver.html
        soup = BeautifulSoup(driver.html, 'html.parser')


        results = soup.find_all('div', id='geoCoordsJsonOutput')
        for result in results:
            result_text = result.text

            result_digits = re.findall('\d+', result_text)
            startswith = [s for s in result_digits if s.startswith('51')] #VA
            fips_elements =[element for element in startswith if len(element)>=15]
            
            try:
                #print(f"{i},{lat[i]},{long[i]} is {fips_elements}")
                geo_id.append(fips_elements)
            except (KeyError, IndexError):
                #print(f"{i},{lat[i]},{long[i]} Not Found")
                not_found.append(i)
                geo_id.append(None)  
            except (TimeoutException):
                driver.navigate.refresh()
    
        #sleep()
        i=i+1

  0%|          | 0/44 [00:00<?, ?it/s]

coordinates wtih more than one return:
385,37.514070000000004,-77.35472 is ['514070000000004', '510872015041003'] 
1178,37.516454700000004,-77.8100575 is ['516454700000004', '511455001022014'] 
2660,37.519462299999994,-77.340384 is ['519462299999994', '510872014061012']
second option reference FIPS code, first option references LatCoord, update fields in DataFrame

https://iqss.github.io/dss-webscrape/filling-in-web-forms.html

In [8]:
geo_id_list =geo_id 

Length Check

In [9]:
print(len(geo_id_list))

44


Matching FIPS to DataFrame

In [10]:
fips_fill = fips_check_scraped_null

In [11]:
fips_fill["FIPS_z"] = geo_id_list
fips_fill

/var/folders/vy/qdp8xcfj4s3dd5c6d1xp_y3w0000gn/T/ipykernel_10032/597263667.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill["FIPS_z"] = geo_id_list


,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
291,4705037110,VA,"7918 Jones Branch Drive, Suite 300, McLean, VA",38.925933,-77.216943,[510594802043017]
326,6940727104,VA,"4625 Creekstone Drive Suite 130, OAKPARK, VA",35.882469,-78.846699,[]
722,1708438404,VA,"7918 Jones Branch Dr Ste 300, McLean, VA",38.925933,-77.216943,[510594802043017]
1246,2219537102,VA,"5534 North MAIN ST, MOUNT JACKSON, VA",38.754798,-78.633557,[511710407001010]
2469,3320167707,VA,"777 6th Street NW, WASHINGTON, VA",38.899370,-77.019590,[]
2653,5527667700,VA,"8200 GREENSBORO DR STE 900, MC LEAN, VA",38.922310,-77.227248,[510594802052013]
2705,3005707104,VA,"8200 Greensboro Dr, Suite 900, MC LEAN, VA",38.922310,-77.227248,[510594802052013]
2927,8472907107,VA,"5534 N MAIN ST, MOUNT JACKSON, VA",38.754798,-78.633557,[511710407001010]
3102,2188557104,VA,"5534 North MAIN ST, MOUNT JACKSON, VA",38.754798,-78.633557,[511710407001010]
3602,1804527302,VA,"8200 Greensboro Drive, Ste 900, MC LEAN, VA",38.922310,-77.227248,[510594802052013]


Remove brackets

In [12]:
fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]
fips_fill

/var/folders/vy/qdp8xcfj4s3dd5c6d1xp_y3w0000gn/T/ipykernel_10032/1219885402.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]


,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
291,4705037110,VA,"7918 Jones Branch Drive, Suite 300, McLean, VA",38.925933,-77.216943,510594802043017
326,6940727104,VA,"4625 Creekstone Drive Suite 130, OAKPARK, VA",35.882469,-78.846699,NaN
722,1708438404,VA,"7918 Jones Branch Dr Ste 300, McLean, VA",38.925933,-77.216943,510594802043017
1246,2219537102,VA,"5534 North MAIN ST, MOUNT JACKSON, VA",38.754798,-78.633557,511710407001010
2469,3320167707,VA,"777 6th Street NW, WASHINGTON, VA",38.899370,-77.019590,NaN
2653,5527667700,VA,"8200 GREENSBORO DR STE 900, MC LEAN, VA",38.922310,-77.227248,510594802052013
2705,3005707104,VA,"8200 Greensboro Dr, Suite 900, MC LEAN, VA",38.922310,-77.227248,510594802052013
2927,8472907107,VA,"5534 N MAIN ST, MOUNT JACKSON, VA",38.754798,-78.633557,511710407001010
3102,2188557104,VA,"5534 North MAIN ST, MOUNT JACKSON, VA",38.754798,-78.633557,511710407001010
3602,1804527302,VA,"8200 Greensboro Drive, Ste 900, MC LEAN, VA",38.922310,-77.227248,510594802052013


In [13]:
fips_fill.loc[fips_fill["FIPS_z"].isnull()] #check #13

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
326,6940727104,VA,"4625 Creekstone Drive Suite 130, OAKPARK, VA",35.882469,-78.846699,NaN
2469,3320167707,VA,"777 6th Street NW, WASHINGTON, VA",38.899370,-77.019590,NaN
4365,3594887305,VA,"1120 Crain Highway North, Chesapeake, VA",39.181778,-76.615519,NaN
4501,4204237701,VA,"55 Joy Drive, South Burlington, VA",44.451222,-73.199114,NaN
5006,2920497210,VA,"1 Spring Street, HOT SPRINGS, VA",34.506051,-93.019824,NaN
6990,9149677006,VA,"4 Wallace Manor Road, BROAD RUN, VA",38.963740,-76.549139,NaN
7834,9682888407,VA,"995 RESEARCH BLVD STE 118, CHARLOTTESVILLE, VA",30.479120,-97.802932,NaN
7930,1800157203,VA,"995 RESEARCH BLVD STE 118, CHARLOTTESVILLE, VA",30.479120,-97.802932,NaN
9677,9453327109,VA,"5518 West Linebaugh Avenue, CHESTER, VA",28.039440,-82.536945,NaN
17181,4222237302,VA,"107 Hillsmere Drive, Annapolis, VA",38.949229,-76.491862,NaN


Parse for Merge

In [14]:
cols= ["LoanNumber", "FIPS_z"]
fips_fill_parsed = fips_fill[cols]
fips_fill_parsed

,LoanNumber,FIPS_z
291,4705037110,510594802043017
326,6940727104,NaN
722,1708438404,510594802043017
1246,2219537102,511710407001010
2469,3320167707,NaN
2653,5527667700,510594802052013
2705,3005707104,510594802052013
2927,8472907107,511710407001010
3102,2188557104,511710407001010
3602,1804527302,510594802052013


Read in base data

In [15]:

#if from FIPS file
fips_check = pd.read_csv("../../../data/state_data/geo/geo_fips/23/VA_fips_scraped.csv") 
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1", "Unnamed: 0"], axis =1) 
fips_check.head()

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,893297,2392497107,VA,"831 INDUSTRIAL AVE, CHESAPEAKE, VA",36.824257,-76.272486,5.155002e+14
1,893298,2649457102,VA,"8827 STAPLES MILL RD, HENRICO, VA",37.633918,-77.507746,5.108720e+14
2,893299,2952437110,VA,"1001 Boulders Parkway Ste 300, RICHMOND, VA",37.511077,-77.528711,5.104110e+14
3,893300,3379937304,VA,"224 D Cornwall Street Suite 403, LEESBURG, VA",39.119430,-77.569008,5.110761e+14
4,893301,3773127208,VA,"2242 W GREAT NECK RD, VIRGINIA BEACH, VA",36.909731,-76.070420,5.181004e+14


Merge with MainFrame

In [16]:
fips_merge = fips_check.merge(fips_fill_parsed, on="LoanNumber", how="left")
fips_merge

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z_x,FIPS_z_y
0,893297,2392497107,VA,"831 INDUSTRIAL AVE, CHESAPEAKE, VA",36.824257,-76.272486,5.155002e+14,NaN
1,893298,2649457102,VA,"8827 STAPLES MILL RD, HENRICO, VA",37.633918,-77.507746,5.108720e+14,NaN
2,893299,2952437110,VA,"1001 Boulders Parkway Ste 300, RICHMOND, VA",37.511077,-77.528711,5.104110e+14,NaN
3,893300,3379937304,VA,"224 D Cornwall Street Suite 403, LEESBURG, VA",39.119430,-77.569008,5.110761e+14,NaN
4,893301,3773127208,VA,"2242 W GREAT NECK RD, VIRGINIA BEACH, VA",36.909731,-76.070420,5.181004e+14,NaN
...,...,...,...,...,...,...,...,...
23219,916516,9512327001,VA,"6000 FORT AVE, LYNCHBURG, VA",37.371058,-79.194838,5.168000e+14,NaN
23220,916517,9677417002,VA,"3337 Emmaus Rd, ROCKINGHAM, VA",38.406665,-78.913133,5.116501e+14,NaN
23221,916518,9715167306,VA,"8503 BUCYRUS CT, MANASSAS, VA",38.761194,-77.454184,5.168391e+14,NaN
23222,916519,9767568404,VA,"554 E Mercury Blvd, Hampton, VA",37.022287,-76.319882,5.165001e+14,NaN


<b>Fill FIPS Columns - if FIPS file

In [17]:
fips_merge["FIPS_z"] = fips_merge["FIPS_z_y"].fillna(fips_merge["FIPS_z_x"])
fips_merge.head()

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z_x,FIPS_z_y,FIPS_z
0,893297,2392497107,VA,"831 INDUSTRIAL AVE, CHESAPEAKE, VA",36.824257,-76.272486,5.155002e+14,NaN,515500203001000.0
1,893298,2649457102,VA,"8827 STAPLES MILL RD, HENRICO, VA",37.633918,-77.507746,5.108720e+14,NaN,510872005033029.0
2,893299,2952437110,VA,"1001 Boulders Parkway Ste 300, RICHMOND, VA",37.511077,-77.528711,5.104110e+14,NaN,510411001071014.0
3,893300,3379937304,VA,"224 D Cornwall Street Suite 403, LEESBURG, VA",39.119430,-77.569008,5.110761e+14,NaN,511076104001030.0
4,893301,3773127208,VA,"2242 W GREAT NECK RD, VIRGINIA BEACH, VA",36.909731,-76.070420,5.181004e+14,NaN,518100430061015.0


Drop Extra FIPS Columns

In [18]:
fips_merge_drop = fips_merge.drop(["FIPS_z_x", "FIPS_z_y"], axis=1)
fips_merge_drop = fips_merge_drop.drop(["Unnamed: 0"], axis=1)
fips_merge_drop

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,2392497107,VA,"831 INDUSTRIAL AVE, CHESAPEAKE, VA",36.824257,-76.272486,515500203001000.0
1,2649457102,VA,"8827 STAPLES MILL RD, HENRICO, VA",37.633918,-77.507746,510872005033029.0
2,2952437110,VA,"1001 Boulders Parkway Ste 300, RICHMOND, VA",37.511077,-77.528711,510411001071014.0
3,3379937304,VA,"224 D Cornwall Street Suite 403, LEESBURG, VA",39.119430,-77.569008,511076104001030.0
4,3773127208,VA,"2242 W GREAT NECK RD, VIRGINIA BEACH, VA",36.909731,-76.070420,518100430061015.0
...,...,...,...,...,...,...
23219,9512327001,VA,"6000 FORT AVE, LYNCHBURG, VA",37.371058,-79.194838,516800009013012.0
23220,9677417002,VA,"3337 Emmaus Rd, ROCKINGHAM, VA",38.406665,-78.913133,511650114005035.0
23221,9715167306,VA,"8503 BUCYRUS CT, MANASSAS, VA",38.761194,-77.454184,516839103011000.0
23222,9767568404,VA,"554 E Mercury Blvd, Hampton, VA",37.022287,-76.319882,516500109001011.0


In [19]:
fips_merge_drop.loc[fips_merge_drop["FIPS_z"].isnull()] #13

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
326,6940727104,VA,"4625 Creekstone Drive Suite 130, OAKPARK, VA",35.882469,-78.846699,NaN
2469,3320167707,VA,"777 6th Street NW, WASHINGTON, VA",38.899370,-77.019590,NaN
4365,3594887305,VA,"1120 Crain Highway North, Chesapeake, VA",39.181778,-76.615519,NaN
4501,4204237701,VA,"55 Joy Drive, South Burlington, VA",44.451222,-73.199114,NaN
5006,2920497210,VA,"1 Spring Street, HOT SPRINGS, VA",34.506051,-93.019824,NaN
6990,9149677006,VA,"4 Wallace Manor Road, BROAD RUN, VA",38.963740,-76.549139,NaN
7834,9682888407,VA,"995 RESEARCH BLVD STE 118, CHARLOTTESVILLE, VA",30.479120,-97.802932,NaN
7930,1800157203,VA,"995 RESEARCH BLVD STE 118, CHARLOTTESVILLE, VA",30.479120,-97.802932,NaN
9677,9453327109,VA,"5518 West Linebaugh Avenue, CHESTER, VA",28.039440,-82.536945,NaN
17181,4222237302,VA,"107 Hillsmere Drive, Annapolis, VA",38.949229,-76.491862,NaN


Convert to File

In [20]:
fips_VA = fips_merge_drop

In [21]:
fips_VA.to_csv("../../../data/state_data/geo/geo_fips/23/VA_fips_scraped.csv") 